### fMRI mini project - HDRf fit for memory task
In this project, we will work on fMRI data from the  Amsterdam Open MRI Collection (AOMIC).

The AOMIC dataset gathers MRI data from more than a thousand individuals obtained on a 3 Tesla imager. For each subject we can access the T1-weighted images ( anatomical image), the diffusion-weighted images ( white-matter tracts)  and fMRI sequences (task-based and resting states). The dataset gives access to both raw and preprocessed (derivative) data. The description of the data acquisition and processing is available here : 

Snoek, L., van der Miesen, M. M., Beemsterboer, T., Van Der Leij, A., Eigenhuis, A., & Scholte, H. S. (2021). The Amsterdam Open MRI Collection, a set of multimodal MRI datasets for individual difference analyses. Scientific data, 8(1), 1-23.

All data are publicly available for downloads using AWS s3 buckets s3://openneuro.org/. The projects will use Jupyter Notebook with the following library : numpy, scipy,  scikit-learn; nilearn.

We will study HRF variation for various task and use convolution tools to fine tune HRF models in different activation task using following model : https://github.com/andrewjahn/AndysBrainBook/blob/master/docs/SPM/SPM_Short_Course/SPM_Statistics/SPM_03_Stats_HRF_Overview.rst 

The AOMIC dataset is avalable for download as S3 bucket. We will use the boto3 package to search and download the elemts we need.
 

In [ ]:
# using pip command, install datalad and git-annex 
!pip ....

From the S3 bucket "openneuro.org"  all links available in https://github.com/OpenNeuroDatasets/dsxxxx. The file and directory strucure corresponds to the path of the object in a [BIDS](https://bids-specification.readthedocs.io/en/stable/index.html) standard.

Find and download the dataset for  PIOP1 cohort (it starts with dsXXX....)


In [ ]:

!datalad clone https://github.com/OpenNeuroDatasets/ds.....


PIOP1 cohort is in directory dsxxx 

preprocessed data are in /derivatives folder

let's get all the task working memory fmri files for  individual sub-001

we are looking for  fmri files for  individual sub-001 that matches  "task-workingmemory" 


In [ ]:
# Loading Resting State images found on AOMIC for ind 0001
##########################################################

# 1 - Filter the data to include task memory only in the preprocessed fmri data
dir_nii=...
task_list=glob.glob(dir_nii+...)
task_list

now list all the nifti images  files, we  can use 'endwith' function 

In [ ]:
nii_task=np.array([x.endswith(...) for x in task_list]) 
nii_files=np.asarray(task_list)[...]
nii_files

select the name for preprocessed workingmemory frmi volumes in MNI space

In [ ]:
file_name = nii_files[...]  # choose the file you want in nii_list    

In [ ]:
# use datalad to actually dowload it

!cd ds002785; datalad get ....

Now we do the same to find the event .tsv file for task working memory the same individual.
you can look for file name that match "events" or "tsv" 


In [ ]:
event_list=glob.glob(dir_nii+...)
event_list=... 
# and print the file list
print (event_list)

In [ ]:
# there should be only one file for working memory
# use datalad to actually dowload it
!cd ds002785; datalad get ....

In [ ]:
# use panda csv reader to load csv file and print it
pd_events=...
pd_events

In [ ]:
# if needed install nilearn package with pip
...

In [ ]:
# import the nilearn image library 
....


In [ ]:
#  read  the tasmemory image volume with nilear image library
img=  ...
#  always check and print the dimension of the data
print(img .... )

In [ ]:
# Now we going to compute the mean (average) image using numpy
# you need to import the numpy library
import ...
# find nilearn image function to compute average 3d volume of a fMRI sequence volume
# compute average image using image library
mean_img= ...
# always check the dimension of the data 
print(.... ) 
# use the  triplanar interactive view of nilearn
# to explore the average 3d volume
... mean_img ...  

Now we are going to extract all the data and plot them separately either as 2d image either as time series.

In [ ]:

from matplotlib import pyplot as plt

# extract 4D array from nilear image object
volume= ....
# check the dimension of the data 
print (volume.shape)
# extract one slice (2D image)
flat_slice=volume..... 
# check thye size
print (flat_slice.shape)
# use matplotlib imshow to plot the slice  
plt.imshow(flat_slice)


In [ ]:
# extract the time serie for one voxel 
ts=volume ....
# in the AOMICS website find repeat time RT : time in seconds between 2 images in [s]  
dt=...
# make a vector of slice times in [s]  
time_vec= ...# vector 
print(time_vec)

In [ ]:
plt.figure(figsize=(14,7))
ax1=plt.subplot(211)
# plot the normalaized time serie
plt.plot(... , label='bold signal at'+np.array2string(np.array([30,35,30])), linewidth=0.5)
plt.ylabel('bold signal')
plt.title('bold signal for 1 voxel')

plt.subplot(212,  sharex = ax1)
# plot the task events as stem of heigth  1
plt.stem() 
# Add title and labels
plt.ylabel('stop task events')
plt.xlabel('time in [s]')
# Add legend
plt.legend()
# Auto space
plt.tight_layout()
# Display plot
plt.show() 

Now we are going to process the time serie. 
First we need to load the scipy libraries

In [ ]:
from scipy.fft import fft,fftfreq
from scipy.fftpack import fftshift
from scipy.signal import spectrogram

## First step  is removing complex signal drift
For that we are fititng three degree polygone curve.

In [ ]:
# a / fit a 4 degree polynome 
# find a0, a, b, c, d that fit the signal y= a0 + ax + bx² +cx³ +dx⁴
# you can use the code used in the filtering exercice 
# withoptimize.curve_fit and a test_func 
# or use numpy.polyfit
(...)=np.polyfit(...)
drift = 

# b/ remove the fitted drift from the signa
yf=


In [ ]:
#plotting the whole process
t = time_vec
y=ts
plt.figure(figsize=(14,10))

# plot signal and the fitted drift
ax2=plt.subplot(411)
plt.plot(t, y, 'b-', label='signal')
plt.plot(t,drift, 'g--', label='drift')
plt.ylabel('bold signal')
plt.legend()
plt.grid()

# plot  sprectrogram for original signal 
freqs, times, spectro = spectrogram( y,fs=1/dt, nperseg=3)
plt.subplot(412, sharex=ax2)
plt.pcolormesh(times, fftshift(freqs), fftshift(spectro, axes=0), shading='gouraud')
plt.ylabel('f [Hz]')
plt.legend()

# plot old and new signal (centerd on the mean)
plt.subplot(413, sharex=ax2)
plt.plot(np.array(range(0,len(ts)))*dt, y-y.mean(), label='old bold signal', linewidth=0.5)
plt.plot(np.array(range(0,len(ts)))*dt, yf-yf.mean(), label='new voxel bold signal', linewidth=2)
plt.legend()

# plot  sprectrogram for new signal 
freqs, times, spectro = spectrogram( yf,fs=1/dt, nperseg=3)
plt.subplot(414, sharex=ax2)
plt.pcolormesh(times, fftshift(freqs), fftshift(spectro, axes=0), shading='gouraud')
plt.ylabel('f [Hz]')
plt.xlabel('t [sec]')
plt.legend()

# Auto space
plt.tight_layout()

# Display plot
plt.show()

Rmoving the drift takes care of very very low frequency noise. 
But some noise may be still present, let's filter it out. 

In [ ]:

# Make a high pass filter with the code used in the filtering exercice 
# with cutoff frequecy described in the paper
from math import pi
import scipy

fc =   # desired cutoff frequency of the filter, Hz
...
# Plot the frequency response ( the code use in the filtering exercice)
...
# apply the filter to the signal obtained after drift removal (the code use in the filtering exercice)
yf_= ...
# plot both the original and filtered signals (the code use in the filtering exercice)
...
# calculate the FFT of the filtered signal and plot the frequency components (the code use in the filtering exercice)
...
# plot spectrogram (the code use in the previous cell )  before and after filtering
...

In [ ]:
# Make a low pass filter  with the code use in the filtering exercice
# with cutoff frequecy described in the paper
fc = ...
...
# Plot the frequency response ( the code use in the filtering exercice)
...
# apply the filter to the signal obtained after drift removal (the code use in the filtering exercice)
yf_= scipy.signal.filtfilt(b, a, yf)
# plot both the original and filtered signals (the code use in the filtering exercice)
...
# calculate the FFT of the filtered signal and plot the frequency components (the code use in the filtering exercice)
...
# plot spectrogram (the code use in the previous cell )  before and after filtering
...

In the nect steps we will try to fit hrf models on our time serie.


In [ ]:
# example of hrf model for a gven onset :
plt.plot(glm.first_level.spm_hrf(tr=1,onset=1))

In [ ]:
# now we are building the hrf model for all are events
# we will use a loop to add 
# what is the propriety of the convolution that 
# makes it possible ?

import nilearn.glm as glm
hrf_mod=  #initialise an empty hrf response vector
for ... in ... : # use all events to add hrf to the model
    hrf= .... # use  glm.first_level.spm_hrf for an event    
    hrf_mod += hrf/sum(hrf) # add it to the model

    plt.plot(time_vec,hrf_mod, label="model HRF ")
plt.ylabel("hrf")
plt.xlabel('t [sec]')
plt.show()

In [ ]:

from scipy import optimize

# Now we will fit our hrf model on the data 
ts =... # choose a time serie for example

# fit an affine model  
# find a0, a, signal y= a0 + ax 
# you can use the code used in the filtering exercice 
# witho ptimize.curve_fit and a function hrf_model_func
def hrf_model_func(...)
    ...

#for a better optimisation we choose stating parameters
p0=...
# we run ; optimize.curve_fit with starting parameters
params, params_covariance =  optimize.curve_fit(...) 

# note : you can use polyfit but you don have acces
# to covariance parameters (by default)


# how do you 
# ingtrpret parameters 
# and covriance parameters  ?
print(params)
print(params_covariance) 

estimates= ...  # build the estimated hrf response 

#plotting the results of the original datapoints and the fitted curve
plt.plot(time_vec, ts, "-r", label = "Bold signal")
plt.plot(time_vec, estimates, label = "Fitted hrf serie")
plt.legend()


In [ ]:
# write a function that implemets all the steps
# run it on the  whole volume (you mightskip voxels that are not in the brain !!),  
# and make a new image with the result